# first project motion detection

### if you watch previous tutorials you will understand what is happening behind syntax briefly, first step, movement is difference between two frames second, difference has noises because of details and light on video so gaussian blurring is eliminating the noises, third, obtaining threshold from clean difference fourth, dilating for eliminating district small weak threshold lines which corrupt healthy threshold detection fifth, finding contours from clean threshold sixth, eliminating small contours which can not be a human by filtering contour area seventh, drawing rectangles for each detected contour on the frame, rectangle dimensions obtained from cv2.boundingRect(contour) that is it


In [5]:
import numpy as np
import cv2

In [13]:
cap = cv2.VideoCapture('vtest.avi')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc('X', 'V', 'I', 'D')
out = cv2.VideoWriter('out.avi', fourcc, 30, (1280,720))

ret, frame1 = cap.read()
ret, frame2 = cap.read()
print(frame1.shape)

while(cap.isOpened()):
    diff = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    
    _, threshold = cv2.threshold(blur, 60, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(threshold, None, iterations=3)        
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)        
#     draw = cv2.drawContours(frame1, contours, -1, (0,255,0), 2)

    for contour in contours:        
        (x, y, w, h) =  cv2.boundingRect(contour)             
        if cv2.contourArea(contour) < 500:
            continue
        cv2.rectangle(frame1, (x, y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(frame1, 'Status: {}'.format('Movement'), (10,20),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
    
        img = cv2.resize(frame1, (1280,720))
        out.write(img)
        cv2.imshow('Motion Detection', frame1)
#         cv2.imshow('threshold', threshold)
#         cv2.imshow('dilated', dilated)
#         cv2.imshow('draw', draw)
    
    frame1 = frame2 # to take all frames in video
    ret, frame2 = cap.read()
    if cv2.waitKey(60) == 27:
        break
    
cv2.destroyAllWindows() # close window

cap.release() # close camera

out.release() # close when write

(576, 768, 3)
